## Mount the Drive, and Change to Google Drive Folder

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount = True)

# %cd /content/drive/MyDrive/MSc.-Dissertations/1/Files
# %ls

## Import Libraries

In [299]:
import tensorflow as tf
from tensorflow import keras
from keras import models, layers, utils, losses
from keras.wrappers import scikit_learn
from keras.models import *
from keras.layers import *
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix

import random
import pandas as pd
import numpy as np
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt

## Count the Number of Files, and Take Random Samples from the Image Files

In [ ]:
# !ls street_view
# count how many files and write the filenames into a file
# !ls street_view -1 | wc -l 
# !ls street_view/*.jpg > flist.txt
flist_old = list(pd.read_csv('flist.txt', header = None)[0])
flist = []
change_names = list(map(lambda x: flist.append(f'street_view/{x}'), flist_old))

# Set seed so sample is reproducible 
# random.seed(99)  # set this to an integer value!!!
# nsamp = 100
# flist_sub = random.sample(flist, nsamp)
# flist = flist_sub

# print(flist)

## Overview of the `properties` Dataset

Read the `properties` dataset first, and make sure that `property type` is a categorical variable.

In [ ]:
properties = pd.read_csv('properties.csv')
properties_juny12 = pd.read_csv('properties_juny12.csv')
properties_full = pd.concat([properties, properties_juny12])
properties = properties_full
properties.propertyType = properties.propertyType.astype('category')
properties.head()

Basic information of the dataset is shown as follows.

In [ ]:
properties.info()

Descriptive statistics of continuous variables are shown as follows.

In [ ]:
properties.describe()

Frequencies of each level of the variable `property type` are obtained as follows.

In [ ]:
properties.propertyType.value_counts()

## A Subset of the `properties` Dataset

As random samples of images have been obtained previously, a subset of the whole `properties` dataset could hence be formulated by selecting the rows of the whole `properties` dataset corresponding to the selected samples.

In [ ]:
flist_id = list(map(lambda string: string[16 : -4], flist))
properties_sub = pd.DataFrame(properties.loc[properties['property_id'].isin(flist_id)])
properties_sub = properties_sub.drop_duplicates(['location_lat', 'location_lng'])
flist_id = list(properties_sub.property_id)
flist_new = []
change_names_new = list(map(lambda x: flist_new.append(f'street_view/gsv_{x}.jpg'), flist_id))

The original data should be splitted into training and testing sets, and the testing set contains 30% of the original data.

In [ ]:
flist_train, flist_test, propertyType_train, propertyType_test = train_test_split(
    flist_new, # image directories
    np.array(properties_sub.propertyType), # property types
    test_size = 0.3)

In [347]:
Img_list_train = list(map(lambda x: np.asarray(Image.open(x).resize((32, 32), Image.LANCZOS)), flist_train))
Img_list_test = list(map(lambda x: np.asarray(Image.open(x).resize((32, 32), Image.LANCZOS)), flist_test))
Img_array_train = np.asarray(Img_list_train)
Img_array_test = np.asarray(Img_list_test)

For categorical variables, one-hot encoder is introduced.

In [ ]:
dummy_propertyType_train = pd.get_dummies(propertyType_train)
dummy_propertyType_test = pd.get_dummies(propertyType_test)
propertyType_test_fac = np.argmax(np.array(dummy_propertyType_test), axis = 1) 

## Multi-Class Classification Using Neural Network

### Multi-Layer Perceptron (MLP) model

In [316]:
def mlp(output_dim):

    '''
    Creates a multi-layer perceptron neural network model without hidden layers.

    Parameter:
    output_dim (int): The number of output classes.
    
    Returns:
    A compiled Keras model.
    '''

    model = Sequential()
    model.add(Rescaling(1. / 255))
    model.add(Flatten())
    model.add(Dense(100, activation = tf.nn.leaky_relu))
    model.add(Dense(output_dim, activation = tf.nn.softmax))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [317]:
MLP = mlp(dummy_propertyType_train.shape[1])
MLP.fit(ImageDataGenerator().flow(Img_array_train, dummy_propertyType_train, batch_size = 64), epochs = 20, batch_size = 20)

Epoch 1/20
170/170 [==============================] - 21s 112ms/step - loss: 4.3988 - accuracy: 0.2638
Epoch 2/20
170/170 [==============================] - 19s 111ms/step - loss: 2.1239 - accuracy: 0.2900
Epoch 3/20
170/170 [==============================] - 20s 114ms/step - loss: 2.2863 - accuracy: 0.2941
Epoch 4/20
170/170 [==============================] - 20s 116ms/step - loss: 2.1181 - accuracy: 0.2961
Epoch 5/20
170/170 [==============================] - 25s 143ms/step - loss: 2.1717 - accuracy: 0.3037
Epoch 6/20
170/170 [==============================] - 18s 108ms/step - loss: 2.0365 - accuracy: 0.3176
Epoch 7/20
170/170 [==============================] - 18s 105ms/step - loss: 2.0035 - accuracy: 0.3140
Epoch 8/20
170/170 [==============================] - 18s 107ms/step - loss: 1.8187 - accuracy: 0.3380
Epoch 9/20
170/170 [==============================] - 18s 105ms/step - loss: 1.8522 - accuracy: 0.3311
Epoch 10/20
170/170 [==============================] - 18s 105ms/step - l

In [318]:
propertyType_test_pred_MLP = np.argmax(MLP.predict(Img_array_test), axis = 1) 
MLP.evaluate(Img_array_test, dummy_propertyType_test)

146/146 [==============================] - 3s 18ms/step - loss: 1.8216 - accuracy: 0.2802


[1.8216115236282349, 0.28024107217788696]

In [319]:
confusion_matrix(propertyType_test_fac, propertyType_test_pred_MLP)

array([[  74,    3,  848,    7,   79],
       [  21,   47,  517,   16,   85],
       [  42,    6, 1047,   13,   82],
       [  34,   22,  988,   43,  156],
       [  13,   10,  393,    9,   91]], dtype=int64)

## Convolutional Neural Network

In [373]:
def cnn(output_dim):

    '''
    Creates a convolutional neural network model without hidden layers.

    Parameter:
    output_dim (int): The number of output classes.
    
    Returns:
    A compiled Keras model.
    '''

    model = Sequential()
    model.add(Rescaling(1. / 255))
    model.add(Conv2D(4, 2, padding = 'same', activation = tf.nn.leaky_relu))
    model.add(MaxPooling2D())  
    model.add(Flatten())
    model.add(Dense(16, activation = tf.nn.leaky_relu))
    model.add(Dense(output_dim, activation = tf.nn.softmax))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [380]:
CNN = cnn(dummy_propertyType_train.shape[1])
CNN.fit(ImageDataGenerator().flow(Img_array_train, dummy_propertyType_train, batch_size = 256), epochs = 20, batch_size = 64)

Epoch 1/20
43/43 [==============================] - 4s 55ms/step - loss: 1.5335 - accuracy: 0.2922
Epoch 2/20
43/43 [==============================] - 2s 55ms/step - loss: 1.4802 - accuracy: 0.3531
Epoch 3/20
43/43 [==============================] - 3s 60ms/step - loss: 1.4600 - accuracy: 0.3715
Epoch 4/20
43/43 [==============================] - 2s 50ms/step - loss: 1.4451 - accuracy: 0.3827
Epoch 5/20
43/43 [==============================] - 2s 49ms/step - loss: 1.4418 - accuracy: 0.3811
Epoch 6/20
43/43 [==============================] - 2s 48ms/step - loss: 1.4269 - accuracy: 0.3913
Epoch 7/20
43/43 [==============================] - 2s 46ms/step - loss: 1.4203 - accuracy: 0.3968
Epoch 8/20
43/43 [==============================] - 2s 46ms/step - loss: 1.4168 - accuracy: 0.3994
Epoch 9/20
43/43 [==============================] - 2s 52ms/step - loss: 1.4091 - accuracy: 0.3986
Epoch 10/20
43/43 [==============================] - 2s 47ms/step - loss: 1.4041 - accuracy: 0.4004
Epoch 11/

In [381]:
propertyType_test_pred_CNN = np.argmax(CNN.predict(Img_array_test), axis = 1) 
CNN.evaluate(Img_array_test, dummy_propertyType_test)

146/146 [==============================] - 2s 11ms/step - loss: 1.4578 - accuracy: 0.3715


[1.4578146934509277, 0.3715023696422577]

In [382]:
confusion_matrix(propertyType_test_fac, propertyType_test_pred_CNN)

array([[417,  49, 418, 112,  15],
       [109, 208, 175, 183,  11],
       [325,  46, 630, 171,  18],
       [216, 116, 449, 445,  17],
       [113,  59, 232,  86,  26]], dtype=int64)

In [ ]:
# import dill
# dill.dump_session('Presetting.pkl')
# # dill.load_session('Presetting.pkl')